In [18]:
import model 
import data 

from importlib import reload
reload(data)
reload(model)

<module 'model' from '/tf/notebooks/PixelWise_Cell_Classification/V2/unet/model.py'>

In [2]:
# for Gunpowder
ZARR_FOLDER = '../../../../data/cells/raw/zarr/train/'
gp_batch_size= 12
gp_voxel_shape = [1,1,1]
gp_input_shape= [256, 256,4]
gp_output_shape = [ 256, 256,4]

# For unet
OUTPUT_PATH = '../../../../data/cells/models/'

unet_input_size = (256,256,4)
unet_output_size = 4

In [3]:
from datetime import date

today = date.today()

d1 = today.strftime("%d_%m_%Y_")
model_name = d1+'unet_v1_cells.hdf5'
model_name

'08_09_2020_unet_v1_cells.hdf5'

### Check data 

In [5]:
gen_train_fast = data.generate_fast_training_batch(ZARR_FOLDER, batch_size=gp_batch_size
                                              ,voxel_shape = gp_voxel_shape,
                                              input_shape= gp_input_shape,output_shape = gp_output_shape )

In [6]:
im,mask = next(gen_train_fast)

In [7]:
test_im = im[0]
test_mask = mask[0]
print('Img size: {} {} < {} | Mask size: {} {} <{}'.format(test_im.shape,test_im.dtype,test_im.max()
                                                                   ,test_mask.shape,test_mask.dtype,test_mask.max()))

Img size: (256, 256, 4) float32 < 0.9843137264251709 | Mask size: (256, 256, 4) float32 <1.0


### Model

In [14]:
unet_model = model.unet(input_size = unet_input_size ,output_size = unet_output_size)

In [16]:
unet_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 4) 0                                            
__________________________________________________________________________________________________
conv2d_69 (Conv2D)              (None, 256, 256, 64) 2368        input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_69[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_12 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_70[0][0]                  
____________________________________________________________________________________________

### Train

In [20]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import os

model_file = os.path.join(OUTPUT_PATH,model_name)
model_checkpoint = ModelCheckpoint(model_file, monitor='mse',verbose=1, save_best_only=True)

In [ ]:
history = unet_model.fit_generator(gen_train_fast,steps_per_epoch=50,epochs=20,callbacks=[model_checkpoint])


Epoch 1/20
50/50 [==============================] - 45s 894ms/step - loss: 0.0569 - mean_io_u: 0.8121
Epoch 2/20
30/50 [=================>............] - ETA: 16s - loss: 0.0415 - mean_io_u: 0.8960

### Test

In [ ]:
im,mask = next(gen_train_fast)

In [ ]:
plt.imshow(im[0])
# print(im[0])

In [ ]:
results = model.predict(im)

In [ ]:
plt.imshow(results[0][:,:,3])